In [2]:
import numpy as np
from tensorflow.keras import regularizers
import math
import tensorflow as tf
import tensorflow.keras.layers as tfl
from keras import backend as K 

In [3]:
from tensorflow.keras.datasets import cifar100

(X_train, y_train), (X_test, y_test) = cifar100.load_data(label_mode='coarse')

In [4]:
print(X_train.shape,y_train.shape)

(50000, 32, 32, 3) (50000, 1)


In [5]:
import cv2
X_train = np.array([cv2.resize(img, (140, 140)) for img in X_train]) 

In [6]:
X_test = np.array([cv2.resize(img, (140, 140)) for img in X_test])

In [7]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
y_train=enc.fit_transform(y_train).toarray().astype(int)
y_test=enc.transform(y_test).toarray().astype(int)


print(y_train.shape)
print(y_train[0])

(50000, 20)
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]


In [8]:
def data_augmenter():
    '''
    Create a Sequential model composed of 2 layers
    Returns:
        tf.keras.Sequential
    '''
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(tfl.RandomFlip('horizontal'))
    data_augmentation.add(tfl.RandomRotation(0.2))

    return data_augmentation

In [9]:
data_augmentation = data_augmenter()

In [10]:
def arch(input_shape, data_augmentation):

    input_img = tf.keras.Input(shape=input_shape)
    layer = data_augmenter()(input_img)
    layer=tfl.Conv2D(filters= 96 , kernel_size= 11,strides=(4, 4))(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(layer)
    layer=tfl.BatchNormalization()(layer,training=True)
    
    layer=tfl.Conv2D(filters= 256 , kernel_size= 5 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.MaxPool2D(pool_size=(3, 3), strides=(2,2), padding='same')(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Conv2D(filters= 384 , kernel_size= 3 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.Conv2D(filters= 384 , kernel_size= 3 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.Conv2D(filters= 256 , kernel_size= 3 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.MaxPool2D(pool_size=(3, 3), strides=(2,2), padding='same')(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Flatten()(layer)
    
    layer=tfl.Dense(units=4096, activation='relu')(layer)
    layer=tfl.Dropout(0.2)(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Dense(units=4096, activation='relu')(layer)
    layer=tfl.Dropout(0.2)(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Dense(units=1000, activation='relu')(layer)
    layer=tfl.Dropout(0.2)(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    outputs=tfl.Dense(units= 20 , activation='softmax')(layer)
    model = tf.keras.Model(inputs=input_img, outputs=outputs)
    return model

In [11]:
conv_model = arch((140, 140, 3),data_augmentation)
conv_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
conv_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 140, 140, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 140, 140, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 33, 33, 96)        34944     
                                                                 
 re_lu (ReLU)                (None, 33, 33, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 17, 17, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 17, 17, 96)       384       
 ormalization)                                               

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(8192)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(8192)
history = conv_model.fit(train_dataset,epochs=10,validation_data=test_dataset,batch_size=8192,shuffle=True)

Epoch 1/10


: 